**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/v3ptdkv5fvmx5zk/iris.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Klasifikácia pomocou umelých neurónových sietí

Tento notebook ukazuje, ako sa dá neurónová sieť zostrojená pomocou pythonového balíčka `PyTorch` aplikovať na jednoduchú klasifikačnú úlohu. Ukážeme, ako sa dá taká sieť vytvoriť a natrénovať. Budeme používať veľmi jednoduchú architektúru – bez konvolučných vrstiev, dávkovej normalizácie a iných podobných špeciálnych vrstiev.

### Načítanie dátovej množiny

V tomto príklade budeme opäť pracovať s dátovou množinou Iris, ktorú už dobre poznáme. Teraz ju načítame z CSV súboru a rozdelíme na tréningovú a testovaciu časť:



In [ ]:
#@title -- Loading and Splitting the dataset df_train, df_test -- { display-mode: "form" }

# we load the data from the CSV
df = pd.read_csv("data/iris.csv")
display(df.head())

# we split it into train and test, stratifying by species
df_train, df_test = train_test_split(df, test_size=0.25,
                                     stratify=df['species'],
                                     random_state=4)

Ako zvyčajne, stĺpce roztriedime na kategorické, numerické a výstupné.



In [ ]:
categorical_inputs = []
numeric_inputs = list(df.columns[:-1])
output = ["species"]

Predspracovanie, ktoré štandardne aplikujeme, prekóduje hodnoty kategorického atribútu na čísla tak, že každej unikátnej hodnote priradí poradové číslo (pomocou transformátora `OrdinalEncoder`). V prípade neurónových sietí môže byť ale vhodnejšie použiť kódovanie 1 z n: každému kategorickému stĺpcu bude zodpovedať toľko vstupných neurónov, koľko rozličných hodnôt kategorická premenná nadobúda a aktívny bude vždy práve jeden z nich. Na takýto typ predspracovania slúži transformátor `OneHotEncoder`. Predspracovanie pre numerické premenné môže zostať nezmenené.

Polia nezabudneme transformovať na PyTorch tenzory s vhodnými dátovými typmi: 32-bitovými floatmi pre vstupy a long int-mi pre označenia tried (výstup). Zároveň si v tomto kroku zvolíme aj zariadenie, presne ako sme to robili aj v predošlých notebook-och a tenzory naň prenesieme.



In [ ]:
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='constant', fill_value='MISSING'),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

In [ ]:
output_preproc = OrdinalEncoder()

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = output_preproc.fit_transform(df_train[output]).reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = output_preproc.transform(df_test[output]).reshape(-1)

device = "cuda" if torch.cuda.is_available() else "cpu"

X_train = torch.as_tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.as_tensor(Y_train, dtype=torch.long).to(device)
X_test = torch.as_tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.as_tensor(Y_test, dtype=torch.long).to(device)

### Vytvorenie neurónovej siete

Naša neurónová sieť bude veľmi podobná tej, ktorú sme používali na regresiu. Počet vstupov sa bude opäť rovnať počtu stĺpcov v našej dátovej množine, zatiaľ čo počet výstupov sa bude odteraz samozrejme rovnať počtu tried, pretože sieť navracia ich príslušné pravdepodobnosti.

Pripomeňme, že v klasifikátoroch vo všeobecnosti používame ako aktivačnú funkciu výstupnej vrstvy funkciu softmax. Táto funkcia zabezpečuje, že súčet výstupov tejto poslednej vrstvy je vždy 1, takže ich možno interpretovať ako správne normalizované pravdepodobnosti. Aplikuje tiež nelineárnu transformáciu, ktorá uľahčuje získanie pravdepodobností blízkych 1.

  **POZOR: V prípade frameworku PyTorch je funkcia softmax súčasťou stratovej funkcie ``nn.CrossEntropyLoss'', preto ju NEPRIDÁVAME NA KONEC MODELU! Poslednú vrstvu necháme lineárnu.** 



In [ ]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_inputs, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, num_outputs)

    def forward(self, x):
        y = self.fc1(x)
        y = torch.relu(y)
        
        y = self.fc2(y)
        y = torch.relu(y)
        
        y = self.fc3(y)
        
        return y

---
### Úloha 1: Tréning siete

**V bunke nižšie dokončite tréningovú slučku a natrénujte neurónovú sieť.** 

Tréningová slučka bude v podstate rovnaká ako pri regresii, s tou výnimkou, že teraz budeme používať `nn.CrossEntropyLoss`.

---


In [ ]:
num_inputs = X_train.shape[1]
num_outputs = len(np.unique(Y_train.cpu()))

model = Net(num_inputs, num_outputs)
model.to(device)

criterion = nn.CrossEntropyLoss()



# ----




In [ ]:
plt.plot(loss_train)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')

### Testovanie

Teraz sme znovu pripravní model hodnotiť. Opäť ho nezabudneme najprv uviesť do evaluačného režimu pomocou `model.eval()` a spustiť ho v rámci `torch.no_grad()`, aby sa zbytočne nevytváral výpočtový graf.

Nezabudnite, že naša sieť predpovedá pravdepodobnosti tried. Aby sme sa z nich dostali k označeniam tried, aplikujeme na pravdepodobnosti funkciu `argmax` (keďže naša sieť neobsahuje záverečnú softmax vrstvu, hodnoty, ktoré tu dostávame, sú v skutočnosti logit-y, nie normalizované pravdepodobnosti, no keď hľadáme maximum, nie je v tom žiadny rozdiel), čím identifikujeme pre každú vzorku najpravdepodobnejšiu triedu.

#### Na tréningových dátach



In [ ]:
model.eval()
with torch.no_grad():
    y_train_logit = model(X_train)
    y_train = y_train_logit.argmax(dim=1)

In [ ]:
Y_train_cpu = Y_train.cpu()
y_train_cpu = y_train.cpu()

cm = pd.crosstab(
    output_preproc.inverse_transform(
        Y_train_cpu.reshape(-1, 1)).reshape(-1),
    output_preproc.inverse_transform(
        y_train_cpu.reshape(-1, 1)).reshape(-1),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm)

In [ ]:
acc = accuracy_score(Y_train_cpu, y_train_cpu)
print("Accuracy = {}".format(acc))

#### Na testovacích dátach



In [ ]:
model.eval()
with torch.no_grad():
    y_test_logit = model(X_test)
    y_test = y_test_logit.argmax(dim=1)

In [ ]:
Y_test_cpu = Y_test.cpu()
y_test_cpu = y_test.cpu()

cm = pd.crosstab(
    output_preproc.inverse_transform(
        Y_test_cpu.reshape(-1, 1)).reshape(-1),
    output_preproc.inverse_transform(
        y_test_cpu.reshape(-1, 1)).reshape(-1),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm)

In [ ]:
acc = accuracy_score(Y_test_cpu, y_test_cpu)
print("Accuracy = {}".format(acc))